In [ ]:
import tweepy
import numpy as np 
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time 
from googletrans import Translator

text = "you're not so smart"
# text = "Great new book just out by the Former Director of Ice, Tom Homan: “Defend the Border and Save Lives: Solving Our Most Important Humanitarian and Security Crisis.” Great guy, get your copy today!"

blob = TextBlob(text)

for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

analyser = SentimentIntensityAnalyzer()
score = analyser.polarity_scores(text)['compound']
print(score)

    

In [ ]:

translator = Translator()
translator.translate('hola, todo bien? vamos! pendeho').text

In [ ]:
def get_sentiment(text):
    trans = translator.translate(text).text 
    score = analyser.polarity_scores(trans)
    return score['compound']

text = "dia orang bodoh."
print(get_sentiment(text))

text = "he's a dumb kid"
print(get_sentiment(text))

text = "eres una perra"
print(get_sentiment(text))


In [ ]:
consumer_key = "ew0pmbxpjcfgMTSZcywA0Fgb7"
consumer_secret = "fO0t6AmR2RvOoAV6aSnVzheDyzCDZz2GJmSWCDsIfjQUXk45fD"
access_token = "1251440497257672705-datdho4HgxFZoj4AkA8c9cmi0skf8b"
access_token_secret = "Gw9RnSjfeMzc4P415gbv494VO33fIPU5uGMCXg8qINLl1"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)


In [ ]:
def list_tweets(user_id, count):
    tweets = api.user_timeline("@" + user_id, count=count, tweet_mode='extended')
    
    for t in tweets:
        print(t.full_text)
        try:
            fav = t.retweeted_status.favorite_count
        except:
            fav = t.favorite_count
        ret = t.retweet_count
    
        print(fav, ret)
        
    return tweets

In [ ]:
count = 5

user_id = 'realDonaldTrump' 
tw_trump = list_tweets(user_id, count)



In [ ]:
print(tw_trump[0].full_text)

In [ ]:

def check_replies(t):
    replies = tweepy.Cursor(api.search, q='to:{}'.format(user_id),
                                    since_id=t.id, tweet_mode='extended').items()
    sentiment = 0
    while True:
        try:
            reply = replies.next()
            sentiment += get_sentiment(reply.full_text)
            
        except StopIteration:
            break

        time.sleep(0.03)
    return sentiment




In [ ]:
class MyStreamListener(tweepy.StreamListener):
    sentiment_score = 0
    
    def on_status(self, status):
        print(status.text)
        try:
            val = get_sentiment(status.text)
            self.sentiment_score +=  val
        except:
            return
        print(self.sentiment_score, end='\r', flush=True)
        
trumpStreamListener = MyStreamListener()
trumpStream = tweepy.Stream(auth = api.auth, listener=trumpStreamListener)
trumpStream.filter(track=['trump'], is_async=False)


In [ ]:
bidenStreamListener = MyStreamListener()
bidenStream = tweepy.Stream(auth = api.auth, listener=bidenStreamListener)
bidenStream.filter(track=['biden'], is_async=False)